In [1]:
# -*-coding: utf-8 -*-
# 对txt文件进行中文分词
import jieba
import os
from utils import files_processing

# 源文件所在目录
source_folder = './three_kingdoms/source'
segment_folder = './three_kingdoms/segment'

In [4]:
#  字词分割，对整个文件内容进行字词分割
def segment_lines(file_list,segment_out_dir,stopwords=[]):
    for i,file in enumerate(file_list):
        segment_out_name=os.path.join(segment_out_dir,'segment_{}.txt'.format(i))
        with open(file, 'rb') as f:
            document = f.read()
            document_cut = jieba.cut(document)
            sentence_segment=[]
            for word in document_cut:
                if word not in stopwords:
                    sentence_segment.append(word)
            result = ' '.join(sentence_segment)
            result = result.encode('utf-8')
            with open(segment_out_name, 'wb') as f2:
                f2.write(result)

# 对source中的txt文件进行分词，输出到segment目录中
file_list=files_processing.get_files_list(source_folder, postfix='*.txt')
segment_lines(file_list, segment_folder)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.198 seconds.
Prefix dict has been built succesfully.


In [12]:
# -*-coding: utf-8 -*-
# 先运行 word_seg进行中文分词，然后再进行word_similarity计算
# 将Word转换成Vec，然后计算相似度 
# !pip install gensim
from gensim.models import word2vec
import multiprocessing

# 如果目录中有多个文件，可以使用PathLineSentences
segment_folder = './three_kingdoms/segment'
sentences = word2vec.PathLineSentences(segment_folder)

# 设置模型参数，进行训练
model = word2vec.Word2Vec(sentences, size=100, window=3, min_count=1)
# print(model.wv.similarity('曹操'))
# print(model.wv.similarity('曹操', '刘备'))
# print(model.wv.most_similar(positive=['曹操', '刘备']))
# print(model.wv.most_similar(positive=['曹操', '刘备'], negative=['张飞']))
# 设置模型参数，进行训练
model2 = word2vec.Word2Vec(sentences, size=128, window=5, min_count=5, workers=multiprocessing.cpu_count())
# 保存模型
model2.save('./models/word2Vec.model')
print(model2.wv.most_similar(positive=['曹操'], topn=10))
print(model2.wv.most_similar(positive=['曹操', '刘备'], negative=['张飞']))


[('孙权', 0.9886712431907654), ('先主', 0.9837744832038879), ('众将', 0.9813681840896606), ('往', 0.9800786972045898), ('夫人', 0.9793328046798706), ('书', 0.9788529872894287), ('孔明谓', 0.9787090420722961), ('袁绍', 0.9783845543861389), ('报', 0.9777935743331909), ('荆州', 0.9775213003158569)]
[('臣', 0.9902024269104004), ('今', 0.9880971908569336), ('丞相', 0.9873654246330261), ('朕', 0.9846106767654419), ('坐守', 0.9829933643341064), ('无忧', 0.982245147228241), ('非', 0.981597900390625), ('陛下', 0.9808411002159119), ('何以', 0.9797478318214417), ('主公', 0.9794127941131592)]
